In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [2]:
df_green = spark.read.parquet('data/raw/green/*/*')

In [3]:
df_green.registerTempTable('green')

In [4]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1
""")

In [5]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 02:00:00| 116|             22.63|             2|
|2020-01-01 02:00:00| 243|             15.29|             1|
|2020-01-01 04:00:00| 223|             36.96|             1|
|2020-01-01 06:00:00| 130|              11.8|             1|
|2020-01-01 07:00:00|  82|              11.8|             1|
|2020-01-01 07:00:00| 116|              6.81|             1|
|2020-01-01 07:00:00|  42|             13.11|             1|
|2020-01-01 07:00:00|  41|27.150000000000002|             3|
|2020-01-01 07:00:00| 179|               7.8|             1|
|2020-01-01 07:00:00| 168|              21.3|             1|
|2020-01-01 07:00:00|  75|              18.3|             1|
|2020-01-01 07:00:00| 248|              11.3|             1|
|2020-01-01 07:00:00| 130|             29.96|             3|
|2020-01-01 08:00:00| 13

In [6]:
df_yellow = spark.read.parquet('data/raw/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [7]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [8]:
df_yellow_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 09:00:00|  24| 783.8999999999999|            44|
|2020-01-01 12:00:00| 202|              42.1|             2|
|2020-01-01 13:00:00| 244|185.17000000000004|            17|
|2020-01-01 13:00:00|  71|             49.71|             1|
|2020-01-01 15:00:00|  48|           2857.21|           120|
|2020-01-01 15:00:00| 179|            119.05|             7|
|2020-01-01 17:00:00|  93|287.46000000000004|             4|
|2020-01-01 18:00:00| 169|              19.1|             2|
|2020-01-01 19:00:00| 193|            105.66|             4|
|2020-01-01 23:00:00|  25|             14.52|             1|
|2020-01-02 03:00:00| 196|             152.8|             2|
|2020-01-02 05:00:00| 157|             61.42|             1|
|2020-01-02 11:00:00| 143| 82.77999999999999|             4|
|2020-01-02 12:00:00| 26

In [9]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [10]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [12]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 09:00:00|  24|126.41999999999999|                   6| 783.8999999999999|                   44|
|2020-01-01 09:00:00| 242|             30.94|                   2|              null|                 null|
|2020-01-01 12:00:00| 202|              null|                null|              42.1|                    2|
|2020-01-01 13:00:00|  71|              null|                null|             49.71|                    1|
|2020-01-01 13:00:00| 244|             25.26|                   2|185.17000000000004|                   17|
|2020-01-01 15:00:00|  48|              null|                null|           2857.21|                  120|
|2020-01-01 15:00:00| 112|  

In [14]:
df_zones = spark.read \
    .option("header", "true") \
    .csv('taxi_zone_lookup.csv')
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [15]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)
df_result.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|LocationID|  Borough|                Zone|service_zone|
+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|2020-01-01 09:00:00|  24|126.41999999999999|                   6| 783.8999999999999|                   44|        24|Manhattan|        Bloomingdale| Yellow Zone|
|2020-01-01 09:00:00| 242|             30.94|                   2|              null|                 null|       242|    Bronx|Van Nest/Morris Park|   Boro Zone|
|2020-01-01 12:00:00| 202|              null|                null|              42.1|                    2|       202|Manhattan|    Roosevelt Island|   Boro Zone|
|2020-01-01 13:00:00| 

In [16]:
df_result.drop('LocationID', 'zone').show()

+-------------------+------------------+--------------------+------------------+---------------------+---------+------------+
|               hour|      green_amount|green_number_records|     yellow_amount|yellow_number_records|  Borough|service_zone|
+-------------------+------------------+--------------------+------------------+---------------------+---------+------------+
|2020-01-01 09:00:00|126.41999999999999|                   6| 783.8999999999999|                   44|Manhattan| Yellow Zone|
|2020-01-01 09:00:00|             30.94|                   2|              null|                 null|    Bronx|   Boro Zone|
|2020-01-01 12:00:00|              null|                null|              42.1|                    2|Manhattan|   Boro Zone|
|2020-01-01 13:00:00|              null|                null|             49.71|                    1| Brooklyn|   Boro Zone|
|2020-01-01 13:00:00|             25.26|                   2|185.17000000000004|                   17|Manhattan|   Bor